# MushCamp EfficientNetB7

### Import Library

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from efficientnet.tfkeras import EfficientNetB7

### Load Data

In [2]:
trainPath = r'dataset\dataSplit\train'
validationPath = r'dataset\dataSplit\val'
testPath = r'dataset\dataSplit\test'

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Data Augmentation

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.45,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

#Train
train_generator = train_datagen.flow_from_directory(
                                            trainPath,
                                                    target_size=(300, 300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                            )

#Validation
validation_generator=val_datagen.flow_from_directory(
                                            validationPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

#Test
test_generator=val_datagen.flow_from_directory(
                                            testPath,
                                                    target_size=(300,300),
                                                    batch_size=64,
                                                    shuffle=True,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                )

Found 2232 images belonging to 9 classes.
Found 279 images belonging to 9 classes.
Found 288 images belonging to 9 classes.


##### CallBacks

In [6]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.4, patience=3, 
                                    verbose=1, mode='min', 
                                    min_delta=0.0001, min_lr=0,
                                    restore_best_weights=True)

##### Load Model EfficientNetB7

In [5]:
pre_trained_model_B7 = EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3),
    pooling='avg',
)

258434480/258434480 [==============================] - 41s 0us/step


##### Layering EfficientNetB7

In [9]:
for layer in pre_trained_model_B7.layers:
    layer.trainable = False


last_layer = pre_trained_model_B7.get_layer('avg_pool')
last_output = last_layer.output


x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense (9, activation='softmax')(x)

model_B7 = Model( pre_trained_model_B7.input, x) 

model_B7.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

##### Training Model with EfficeintNetB7

In [10]:
history = model_B7.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[reduceLROnPlat],
            verbose = 1)  

Epoch 1/20
35/35 [==============================] - 77s 2s/step - loss: 1.7109 - accuracy: 0.3849 - val_loss: 1.1633 - val_accuracy: 0.6093 - lr: 0.0010
Epoch 2/20
35/35 [==============================] - 45s 1s/step - loss: 1.2898 - accuracy: 0.5435 - val_loss: 0.8311 - val_accuracy: 0.7455 - lr: 0.0010
Epoch 3/20
35/35 [==============================] - 47s 1s/step - loss: 1.1015 - accuracy: 0.6129 - val_loss: 0.7037 - val_accuracy: 0.7634 - lr: 0.0010
Epoch 4/20
35/35 [==============================] - 47s 1s/step - loss: 0.9582 - accuracy: 0.6640 - val_loss: 0.6803 - val_accuracy: 0.7563 - lr: 0.0010
Epoch 5/20
35/35 [==============================] - 46s 1s/step - loss: 0.9489 - accuracy: 0.6788 - val_loss: 0.5992 - val_accuracy: 0.7993 - lr: 0.0010
Epoch 6/20
35/35 [==============================] - 47s 1s/step - loss: 0.8578 - accuracy: 0.7056 - val_loss: 0.6106 - val_accuracy: 0.7814 - lr: 0.0010
Epoch 7/20
35/35 [==============================] - 48s 1s/step - loss: 0.8166 - a

##### Eval Model using Data Testing

In [11]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

82.0

##### Prediction Image B7

In [ ]:
from keras.models import load_model
import keras.utils as image
import matplotlib.pyplot as plt
import numpy as np

model = load_model('Saved_Models/model_EfficientNetB7.h5')

In [ ]:
B7_eval = model_B7.evaluate(test_generator,verbose=0)
B7_acc = round(B7_eval[1],2) * 100
B7_acc

In [ ]:
img_path = ""
CLASS_NAMES = ['Agglonema', 'Alocasia', 'Gelombang Cinta', 'Janda Bolong', 'Lidah Mertua', 'Lili Paris', 'Pucuk Merah', 'Suplir']

img = image.load_img(img_path, target_size=(275,275))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10,verbose=0)
print(f'Hasil Prediksi: {CLASS_NAMES[np.argmax(classes)]}')